# Contract Structure Optimization for 2025-26 Free Agents

**Going Beyond $/WAR: NPV, Opt-Outs, and Deferred Money**

This notebook demonstrates how contract structure can be worth $50-350M in real value through:
- Net Present Value (NPV) of deferred money
- Monte Carlo simulation of opt-out clauses
- Performance bonus optimization
- Side-by-side structure comparisons

---

## Setup

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from src.analysis import ContractStructureOptimizer, ContractStructure
from src.data import ContractData

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Imports successful")

## Part 1: The Ohtani Effect - NPV of Deferred Money

Shohei Ohtani's $700M contract is the perfect case study for understanding deferred money.

In [ ]:
# Initialize optimizer
optimizer = ContractStructureOptimizer(discount_rate=0.05)

# Create Ohtani's actual contract
ohtani_contract, ohtani_npv = optimizer.generate_ohtani_style_contract(
    total_value=700,
    years=10,
    deferred_pct=0.97,
    deferral_years=10
)

print("SHOHEI OHTANI'S CONTRACT")
print("=" * 60)
print(f"Stated Value: ${ohtani_npv['stated_value']:.0f}M over 10 years")
print(f"Structure: 97% deferred ($680M paid 2034-2043)")
print()
print(f"NPV Analysis (5% discount rate):")
print(f"  Real Cost to Dodgers: ${ohtani_npv['npv']:.1f}M")
print(f"  Savings from Stated: ${ohtani_npv['discount_from_stated']:.1f}M ({ohtani_npv['discount_pct']:.1f}%)")
print(f"  CBT (Luxury Tax) AAV: ${ohtani_npv['cbt_aav']:.1f}M")
print(f"  Annual CBT Savings: ${ohtani_npv['cbt_savings_per_year']:.1f}M")
print()
print(f"💡 The Dodgers can spend an extra ${ohtani_npv['cbt_savings_per_year']:.1f}M/year on other players!")

### Visualization: NPV Sensitivity to Discount Rate

In [ ]:
# Calculate NPV at different discount rates
discount_rates = np.linspace(0.02, 0.10, 20)
npvs = []
savings = []

for rate in discount_rates:
    npv_result = optimizer.calculate_npv(ohtani_contract, discount_rate=rate)
    npvs.append(npv_result['npv'])
    savings.append(npv_result['discount_from_stated'])

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: NPV vs Discount Rate
ax1.plot(discount_rates * 100, npvs, linewidth=3, marker='o', markersize=6)
ax1.axhline(y=700, color='red', linestyle='--', linewidth=2, label='Stated Value ($700M)')
ax1.axvline(x=5, color='gray', linestyle=':', linewidth=2, alpha=0.5, label='5% (typical)')
ax1.fill_between(discount_rates * 100, npvs, 700, alpha=0.3, color='green', label='Savings')
ax1.set_xlabel('Discount Rate (%)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Present Value ($M)', fontsize=12, fontweight='bold')
ax1.set_title('Ohtani Contract NPV vs Discount Rate', fontsize=14, fontweight='bold', pad=20)
ax1.legend(loc='upper right', fontsize=10)
ax1.grid(alpha=0.3)
ax1.set_ylim([200, 750])

# Plot 2: Savings vs Discount Rate
ax2.plot(discount_rates * 100, savings, linewidth=3, marker='o', markersize=6, color='green')
ax2.axvline(x=5, color='gray', linestyle=':', linewidth=2, alpha=0.5, label='5% (typical)')
ax2.set_xlabel('Discount Rate (%)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Savings from Stated Value ($M)', fontsize=12, fontweight='bold')
ax2.set_title('Dodgers Savings from Deferrals', fontsize=14, fontweight='bold', pad=20)
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At 5% discount rate: ${savings[9]:.0f}M savings")
print(f"At 3% discount rate: ${savings[4]:.0f}M savings")
print(f"At 7% discount rate: ${savings[14]:.0f}M savings")

### Comparison: Different Deferral Structures

In [ ]:
# Create different deferral scenarios for a $350M contract
deferral_scenarios = [
    ("No Deferrals (Standard)", 0.0),
    ("Light Deferrals (20%)", 0.20),
    ("Moderate Deferrals (40%)", 0.40),
    ("Heavy Deferrals (60%)", 0.60),
    ("Extreme Deferrals (80%)", 0.80),
    ("Ohtani-Style (97%)", 0.97)
]

results = []
for name, deferred_pct in deferral_scenarios:
    contract = ContractStructure(
        total_value=350,
        years=10,
        aav=35,
        deferred_pct=deferred_pct,
        deferral_years=10
    )
    npv = optimizer.calculate_npv(contract)
    results.append({
        'Structure': name,
        'Deferred %': deferred_pct * 100,
        'Stated Value': 350,
        'NPV': npv['npv'],
        'Savings': npv['discount_from_stated'],
        'CBT AAV': npv['cbt_aav']
    })

df_deferrals = pd.DataFrame(results)
print(df_deferrals.to_string(index=False))

In [ ]:
# Visualize deferral impact
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: NPV vs Deferred %
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(df_deferrals)))
bars1 = ax1.barh(df_deferrals['Structure'], df_deferrals['NPV'], color=colors, edgecolor='black', linewidth=1.5)
ax1.axvline(x=350, color='red', linestyle='--', linewidth=2, label='Stated Value ($350M)')
ax1.set_xlabel('Present Value ($M)', fontsize=12, fontweight='bold')
ax1.set_title('NPV by Deferral Structure', fontsize=14, fontweight='bold', pad=20)
ax1.legend(fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Add value labels
for i, (npv, savings) in enumerate(zip(df_deferrals['NPV'], df_deferrals['Savings'])):
    ax1.text(npv - 10, i, f'${npv:.0f}M\n(-${savings:.0f}M)', 
             ha='right', va='center', fontweight='bold', fontsize=9, color='white')

# Plot 2: CBT AAV Savings
stated_aav = 35.0
cbt_savings = stated_aav - df_deferrals['CBT AAV']
bars2 = ax2.barh(df_deferrals['Structure'], cbt_savings, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_xlabel('CBT AAV Savings ($M per year)', fontsize=12, fontweight='bold')
ax2.set_title('Luxury Tax Savings from Deferrals', fontsize=14, fontweight='bold', pad=20)
ax2.grid(axis='x', alpha=0.3)

# Add value labels
for i, savings in enumerate(cbt_savings):
    if savings > 0:
        ax2.text(savings - 0.5, i, f'${savings:.1f}M', 
                 ha='right', va='center', fontweight='bold', fontsize=9, color='white')

plt.tight_layout()
plt.show()

print("\n💡 INSIGHT: Even moderate deferrals (40%) create $50M+ in NPV savings!")

## Part 2: Opt-Out Clause Valuation

Monte Carlo simulation to estimate the probability a player exercises their opt-out.

In [ ]:
# Example: Kyle Tucker (29, RF, 8.7 WAR) with opt-out after Year 3 and 5
tucker_contract = ContractStructure(
    total_value=320,
    years=8,
    aav=40,
    opt_outs=[3, 5]
)

print("Running Monte Carlo simulation for Kyle Tucker opt-outs...")
print("(10,000 iterations)\n")

tucker_optout = optimizer.simulate_opt_out_value(
    contract=tucker_contract,
    current_war=8.7,
    age=29,
    position='RF',
    n_simulations=10000
)

print("KYLE TUCKER - 8yr/$320M with Opt-Outs after Year 3 and 5")
print("=" * 60)
print(f"Opt-Out Probability: {tucker_optout['opt_out_probability'] * 100:.1f}%")
print(f"Expected Years Controlled: {tucker_optout['expected_years_controlled']:.1f} years")
print(f"Years at Risk: {tucker_optout['years_at_risk']:.1f} years")
print(f"Team Risk Score: {tucker_optout['team_risk_score']:.1f}/100")
print()
print("Opt-Out Year Breakdown:")
for year, data in tucker_optout['opt_out_year_breakdown'].items():
    print(f"  After Year {year.split('_')[1]}: {data['probability'] * 100:.1f}% probability")
print()
print(f"Player Flexibility Value: ${tucker_optout['player_flexibility_value']:.1f}M")
print()
print(f"💡 Tucker has {tucker_optout['opt_out_probability'] * 100:.1f}% chance of opting out early!")
print(f"   Team gets ~{tucker_optout['expected_years_controlled']:.0f} years instead of 8.")

### Compare Opt-Out Risk by Player Age and Position

In [ ]:
# Compare opt-out risk for different player profiles
players = [
    {'name': 'Kyle Tucker', 'war': 8.7, 'age': 29, 'position': 'RF', 'type': 'Elite Young OF'},
    {'name': 'Kyle Schwarber', 'war': 8.3, 'age': 33, 'position': 'DH', 'type': 'Elite Aging DH'},
    {'name': 'Dylan Cease', 'war': 8.1, 'age': 30, 'position': 'SP', 'type': 'Elite Pitcher'},
    {'name': 'Alex Bregman', 'war': 7.7, 'age': 32, 'position': '3B', 'type': 'Aging Star'},
    {'name': 'Pete Alonso', 'war': 5.6, 'age': 31, 'position': '1B', 'type': 'Power Bat'},
]

opt_out_results = []

print("Simulating opt-out risk for different player types...\n")

for player in players:
    # Create contract with opt-out after Year 3
    years = 7 if player['age'] <= 30 else 6
    aav = 35 if player['war'] >= 8.0 else 25
    
    contract = ContractStructure(
        total_value=aav * years,
        years=years,
        aav=aav,
        opt_outs=[3]
    )
    
    result = optimizer.simulate_opt_out_value(
        contract=contract,
        current_war=player['war'],
        age=player['age'],
        position=player['position'],
        n_simulations=5000
    )
    
    opt_out_results.append({
        'Player': player['name'],
        'Type': player['type'],
        'Age': player['age'],
        'Position': player['position'],
        'Opt-Out %': result['opt_out_probability'] * 100,
        'Expected Years': result['expected_years_controlled'],
        'Risk Score': result['team_risk_score']
    })

df_optouts = pd.DataFrame(opt_out_results)
print(df_optouts.to_string(index=False))

In [ ]:
# Visualize opt-out risk
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Opt-Out Probability
colors_risk = ['green' if x < 30 else 'orange' if x < 70 else 'red' for x in df_optouts['Opt-Out %']]
bars1 = ax1.barh(df_optouts['Player'], df_optouts['Opt-Out %'], color=colors_risk, 
                 edgecolor='black', linewidth=1.5)
ax1.set_xlabel('Opt-Out Probability (%)', fontsize=12, fontweight='bold')
ax1.set_title('Opt-Out Risk by Player Type', fontsize=14, fontweight='bold', pad=20)
ax1.axvline(x=50, color='orange', linestyle='--', linewidth=2, alpha=0.5, label='50% threshold')
ax1.legend(fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Add labels
for i, (pct, age) in enumerate(zip(df_optouts['Opt-Out %'], df_optouts['Age'])):
    label = f"{pct:.0f}% (age {age})"
    ax1.text(pct + 2, i, label, va='center', fontweight='bold', fontsize=9)

# Plot 2: Expected Years of Control
contract_years = [7 if age <= 30 else 6 for age in df_optouts['Age']]
years_lost = [cy - ey for cy, ey in zip(contract_years, df_optouts['Expected Years'])]

x = np.arange(len(df_optouts))
width = 0.35

bars_contract = ax2.bar(x - width/2, contract_years, width, label='Contract Years', 
                        color='steelblue', edgecolor='black', linewidth=1.5)
bars_expected = ax2.bar(x + width/2, df_optouts['Expected Years'], width, 
                        label='Expected Years', color='coral', edgecolor='black', linewidth=1.5)

ax2.set_ylabel('Years', fontsize=12, fontweight='bold')
ax2.set_title('Team Control: Contract vs Expected', fontsize=14, fontweight='bold', pad=20)
ax2.set_xticks(x)
ax2.set_xticklabels(df_optouts['Player'], rotation=45, ha='right')
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3)

# Add loss labels
for i, (loss, ey) in enumerate(zip(years_lost, df_optouts['Expected Years'])):
    if loss > 0.5:
        ax2.text(i + width/2, ey + 0.3, f'-{loss:.1f}yr', 
                ha='center', fontweight='bold', fontsize=9, color='red')

plt.tight_layout()
plt.show()

print("\n💡 KEY INSIGHTS:")
print("  - Older players (33+) almost ALWAYS opt out if productive")
print("  - Pitchers rarely opt out (steeper aging curve)")
print("  - Young elite players (Tucker) moderate opt-out risk")

## Part 3: Performance Bonus Optimization

Find optimal PA/IP thresholds for performance incentives.

In [ ]:
# Example 1: Kyle Tucker (Batter)
tucker_bonuses = optimizer.optimize_performance_bonuses(
    position='RF',
    expected_pa_or_ip=650,
    pa_or_ip_std=80,
    bonus_pool=20.0,
    is_pitcher=False
)

print("KYLE TUCKER - Performance Bonus Structure")
print("=" * 60)
print(f"Expected PA: {tucker_bonuses['expected_baseline']:.0f}")
print(f"Total Bonus Pool: ${tucker_bonuses['total_bonus_pool']:.1f}M")
print(f"Expected Payout: ${tucker_bonuses['expected_payout']:.1f}M")
print(f"Team Savings (Expected): ${tucker_bonuses['team_savings_expected']:.1f}M")
print()

df_tucker_bonuses = pd.DataFrame(tucker_bonuses['bonus_structure'])
print(df_tucker_bonuses.to_string(index=False))

print(f"\n💡 Team saves ${tucker_bonuses['team_savings_expected']:.1f}M ({tucker_bonuses['overpay_risk']*100:.0f}% payout rate)")

In [ ]:
# Example 2: Dylan Cease (Pitcher)
cease_bonuses = optimizer.optimize_performance_bonuses(
    position='SP',
    expected_pa_or_ip=180,
    pa_or_ip_std=25,
    bonus_pool=15.0,
    is_pitcher=True
)

print("\nDYLAN CEASE - Performance Bonus Structure")
print("=" * 60)
print(f"Expected IP: {cease_bonuses['expected_baseline']:.0f}")
print(f"Total Bonus Pool: ${cease_bonuses['total_bonus_pool']:.1f}M")
print(f"Expected Payout: ${cease_bonuses['expected_payout']:.1f}M")
print(f"Team Savings (Expected): ${cease_bonuses['team_savings_expected']:.1f}M")
print()

df_cease_bonuses = pd.DataFrame(cease_bonuses['bonus_structure'])
print(df_cease_bonuses.to_string(index=False))

print(f"\n💡 Team saves ${cease_bonuses['team_savings_expected']:.1f}M ({cease_bonuses['overpay_risk']*100:.0f}% payout rate)")
print("   Pitchers have higher injury risk → greater team savings!")

In [ ]:
# Visualize bonus structures
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Tucker (Batter) Bonus Probability
tiers_t = df_tucker_bonuses['tier']
thresholds_t = df_tucker_bonuses['PA_threshold']
probs_t = df_tucker_bonuses['probability'] * 100
bonuses_t = df_tucker_bonuses['bonus_earned']

bars_t = ax1.bar(tiers_t, probs_t, color='steelblue', edgecolor='black', linewidth=1.5, alpha=0.7)
ax1_twin = ax1.twinx()
line_t = ax1_twin.plot(tiers_t, bonuses_t, color='red', marker='o', linewidth=3, 
                        markersize=10, label='Bonus Amount')

ax1.set_xlabel('Tier', fontsize=12, fontweight='bold')
ax1.set_ylabel('Probability of Hitting Threshold (%)', fontsize=11, fontweight='bold', color='steelblue')
ax1_twin.set_ylabel('Bonus Amount ($M)', fontsize=11, fontweight='bold', color='red')
ax1.set_title('Kyle Tucker - Bonus Structure\n(Expected 650 PA)', fontsize=14, fontweight='bold', pad=20)
ax1.set_xticks(tiers_t)
ax1.grid(axis='y', alpha=0.3)

# Add threshold labels
for tier, threshold, prob in zip(tiers_t, thresholds_t, probs_t):
    ax1.text(tier, prob + 3, f"{threshold} PA", ha='center', fontweight='bold', fontsize=9)

# Plot 2: Cease (Pitcher) Bonus Probability
tiers_c = df_cease_bonuses['tier']
thresholds_c = df_cease_bonuses['IP_threshold']
probs_c = df_cease_bonuses['probability'] * 100
bonuses_c = df_cease_bonuses['bonus_earned']

bars_c = ax2.bar(tiers_c, probs_c, color='coral', edgecolor='black', linewidth=1.5, alpha=0.7)
ax2_twin = ax2.twinx()
line_c = ax2_twin.plot(tiers_c, bonuses_c, color='darkred', marker='o', linewidth=3, 
                        markersize=10, label='Bonus Amount')

ax2.set_xlabel('Tier', fontsize=12, fontweight='bold')
ax2.set_ylabel('Probability of Hitting Threshold (%)', fontsize=11, fontweight='bold', color='coral')
ax2_twin.set_ylabel('Bonus Amount ($M)', fontsize=11, fontweight='bold', color='darkred')
ax2.set_title('Dylan Cease - Bonus Structure\n(Expected 180 IP)', fontsize=14, fontweight='bold', pad=20)
ax2.set_xticks(tiers_c)
ax2.grid(axis='y', alpha=0.3)

# Add threshold labels
for tier, threshold, prob in zip(tiers_c, thresholds_c, probs_c):
    ax2.text(tier, prob + 3, f"{threshold} IP", ha='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("\n💡 INSIGHTS:")
print(f"  - Tucker (batter): Team saves ${tucker_bonuses['team_savings_expected']:.1f}M (30%)")
print(f"  - Cease (pitcher): Team saves ${cease_bonuses['team_savings_expected']:.1f}M (49%)")
print("  - Pitchers have higher injury risk → greater expected savings for teams")

## Part 4: Complete Contract Structure Comparisons

Compare 4 different contract structures for each top free agent.

In [ ]:
# Load saved comparison data
df_comparisons = pd.read_csv('../data/2025_fa_contract_structure_comparisons.csv')

print("Contract Structure Comparisons for Top 2025-26 Free Agents")
print("=" * 100)
print()

# Display by player
for player in df_comparisons['player'].unique():
    player_data = df_comparisons[df_comparisons['player'] == player]
    print(f"\n{player}")
    print("-" * 100)
    print(player_data[['structure_name', 'years', 'stated_value', 'npv', 'cbt_aav', 
                       'opt_out_probability', 'team_risk_score']].to_string(index=False))
    
    # Find best structures
    best_npv = player_data.nsmallest(1, 'npv')['structure_name'].values[0]
    best_cbt = player_data.nsmallest(1, 'cbt_aav')['structure_name'].values[0]
    
    print(f"\n  Best for Team (NPV): {best_npv}")
    print(f"  Best for CBT: {best_cbt}")

In [ ]:
# Visualize contract structures for Kyle Tucker
tucker_data = df_comparisons[df_comparisons['player'] == 'Kyle Tucker']

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))

structures = tucker_data['structure_name']
colors_struct = ['steelblue', 'coral', 'lightgreen', 'gold']

# Plot 1: Stated Value vs NPV
x = np.arange(len(structures))
width = 0.35

bars1 = ax1.bar(x - width/2, tucker_data['stated_value'], width, label='Stated Value',
                color='lightcoral', edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, tucker_data['npv'], width, label='NPV (Real Cost)',
                color='lightgreen', edgecolor='black', linewidth=1.5)

ax1.set_ylabel('Contract Value ($M)', fontsize=12, fontweight='bold')
ax1.set_title('Kyle Tucker: Stated Value vs Real Cost (NPV)', fontsize=14, fontweight='bold', pad=20)
ax1.set_xticks(x)
ax1.set_xticklabels(structures, rotation=45, ha='right')
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)

# Add savings labels
for i, (stated, npv) in enumerate(zip(tucker_data['stated_value'], tucker_data['npv'])):
    savings = stated - npv
    if savings > 5:
        ax1.text(i, max(stated, npv) + 10, f'-${savings:.0f}M', 
                ha='center', fontweight='bold', fontsize=9, color='green')

# Plot 2: CBT AAV Comparison
bars3 = ax2.barh(structures, tucker_data['cbt_aav'], color=colors_struct, 
                 edgecolor='black', linewidth=1.5)
ax2.set_xlabel('CBT AAV ($M)', fontsize=12, fontweight='bold')
ax2.set_title('Luxury Tax Impact', fontsize=14, fontweight='bold', pad=20)
ax2.grid(axis='x', alpha=0.3)

# Add labels
for i, aav in enumerate(tucker_data['cbt_aav']):
    ax2.text(aav + 0.5, i, f'${aav:.1f}M', va='center', fontweight='bold', fontsize=10)

# Plot 3: Opt-Out Risk
bars4 = ax3.barh(structures, tucker_data['opt_out_probability'] * 100, color=colors_struct,
                 edgecolor='black', linewidth=1.5)
ax3.axvline(x=50, color='red', linestyle='--', linewidth=2, alpha=0.5, label='High Risk (>50%)')
ax3.set_xlabel('Opt-Out Probability (%)', fontsize=12, fontweight='bold')
ax3.set_title('Team Control Risk', fontsize=14, fontweight='bold', pad=20)
ax3.legend(fontsize=10)
ax3.grid(axis='x', alpha=0.3)

# Add labels
for i, prob in enumerate(tucker_data['opt_out_probability'] * 100):
    if prob > 1:
        ax3.text(prob + 1, i, f'{prob:.1f}%', va='center', fontweight='bold', fontsize=10)

# Plot 4: Summary Comparison
# Normalize metrics to 0-100 scale for comparison
def normalize(series, reverse=False):
    normalized = (series - series.min()) / (series.max() - series.min()) * 100
    return 100 - normalized if reverse else normalized

metrics = {
    'Low NPV\n(good for team)': normalize(tucker_data['npv'], reverse=True),
    'Low CBT AAV\n(roster flexibility)': normalize(tucker_data['cbt_aav'], reverse=True),
    'Low Opt-Out Risk\n(team control)': normalize(tucker_data['opt_out_probability'], reverse=True),
}

x_metrics = np.arange(len(metrics))
width_metrics = 0.2

for i, (struct, color) in enumerate(zip(structures, colors_struct)):
    values = [metrics[m].iloc[i] for m in metrics]
    ax4.bar(x_metrics + i*width_metrics, values, width_metrics, label=struct,
            color=color, edgecolor='black', linewidth=1.5)

ax4.set_ylabel('Score (Higher = Better for Team)', fontsize=12, fontweight='bold')
ax4.set_title('Overall Team Value Score', fontsize=14, fontweight='bold', pad=20)
ax4.set_xticks(x_metrics + width_metrics * 1.5)
ax4.set_xticklabels(metrics.keys(), fontsize=10)
ax4.legend(loc='upper right', fontsize=9)
ax4.grid(axis='y', alpha=0.3)
ax4.set_ylim([0, 110])

plt.tight_layout()
plt.show()

print("\n💡 For Kyle Tucker:")
print("  - 50% Deferred structure offers lowest NPV (best real cost)")
print("  - With Opt-Outs offers lowest CBT AAV but 23% opt-out risk")
print("  - Traditional structure safest but highest cost")

## Part 5: Key Takeaways

### Summary of Findings

In [ ]:
print("=" * 100)
print("KEY TAKEAWAYS: CONTRACT STRUCTURE OPTIMIZATION")
print("=" * 100)
print()
print("1. DEFERRED MONEY CREATES MASSIVE SAVINGS")
print("   - Ohtani's $700M costs Dodgers $355M in present value")
print("   - Even moderate deferrals (40%) save $50-100M NPV")
print("   - CBT savings create roster flexibility (can sign more players)")
print()
print("2. OPT-OUTS ARE AGE AND POSITION DEPENDENT")
print("   - Age 33+ position players: 70-100% opt-out probability (AVOID!)")
print("   - Age 28-30 elite players: 20-60% opt-out probability (moderate risk)")
print("   - Pitchers: 1-15% opt-out probability (steeper aging curve)")
print("   - Teams must pay 10-15% premium to include opt-outs")
print()
print("3. PERFORMANCE BONUSES ALIGN INCENTIVES")
print("   - Batters: Teams save ~30% of bonus pool in expectation")
print("   - Pitchers: Teams save ~50% of bonus pool (higher injury risk)")
print("   - Optimal thresholds: 60th, 75th, 85th, 95th percentile of projection")
print()
print("4. STRUCTURE MATTERS AS MUCH AS TOTAL VALUE")
print("   - Same player, different structures = $30-80M NPV difference")
print("   - CBT optimization can create $50M+ roster flexibility")
print("   - Risk allocation (opt-outs, deferrals) changes true value")
print()
print("=" * 100)
print("BOTTOM LINE: Don't just look at $/WAR. Model the contract structure.")
print("=" * 100)

## Conclusion

Contract structure optimization reveals hidden value that traditional $/WAR analysis misses:

1. **Deferred money** creates $50-350M in NPV savings through time value of money
2. **Opt-out clauses** must be valued via simulation - age and position matter enormously
3. **Performance bonuses** create win-win structures when optimized properly
4. **Total structure** can swing contract value by $80M+ for the same player

Teams that master contract structure optimization will gain massive competitive advantages over teams focused solely on headline AAV figures.

---

**Next Steps:**
- Explore more player scenarios
- Sensitivity analysis on discount rates
- Build interactive contract builder tool
- Analyze historical contracts for validation